In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

2025-04-25 17:36:14.888679: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-25 17:36:14.889153: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-25 17:36:14.891006: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-25 17:36:14.896351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745582774.905056  472562 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745582774.90

## load the dataset 

In [2]:
train = pd.read_csv(r'/home/litzchill/vamshi/Podcast_listen_time/dataset/train.csv')

# test = pd.read_csv(r'C:\Users\Vamshi\Desktop\Kaggle_competitions\playground-series\dataset\test.csv')

In [3]:
train

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [4]:
train_df=pd.DataFrame(train)

In [5]:
train_df['Episode_Length_minutes'] = train_df['Episode_Length_minutes'].fillna(train_df['Episode_Length_minutes'].median())
train_df['Guest_Popularity_percentage'] = train_df['Guest_Popularity_percentage'].fillna(train_df['Guest_Popularity_percentage'].median())
train_df['Number_of_Ads'] = train_df['Number_of_Ads'].fillna(train_df['Number_of_Ads'].median())

In [6]:
train_df.isnull().sum()

id                             0
Podcast_Name                   0
Episode_Title                  0
Episode_Length_minutes         0
Genre                          0
Host_Popularity_percentage     0
Publication_Day                0
Publication_Time               0
Guest_Popularity_percentage    0
Number_of_Ads                  0
Episode_Sentiment              0
Listening_Time_minutes         0
dtype: int64

In [7]:
train_df.drop(columns=['id'], inplace=True)

In [8]:
numeric_cols = train_df.select_dtypes(include='number').columns

for col in numeric_cols:
    zero_count = (train_df[col] == 0).sum()
    print(f"Number of zero values in '{col}': {zero_count}")

Number of zero values in 'Episode_Length_minutes': 1
Number of zero values in 'Host_Popularity_percentage': 0
Number of zero values in 'Guest_Popularity_percentage': 3
Number of zero values in 'Number_of_Ads': 217592
Number of zero values in 'Listening_Time_minutes': 8551


In [9]:
import numpy as np

# Columns where 0 likely means missing
cols_to_replace = ['Episode_Length_minutes', 'Host_Popularity_percentage', 
                   'Guest_Popularity_percentage', 'Listening_Time_minutes']

# Replace 0s with NaNs
train_df[cols_to_replace] = train_df[cols_to_replace].replace(0, np.nan)

# Impute NaNs with median
for col in cols_to_replace:
    train_df[col] = train_df[col].fillna(train_df[col].mean())



In [10]:
# define numerical & categorical columns
numeric_features = [feature for feature in train_df.columns if train_df[feature].dtype != 'O']
categorical_features = [feature for feature in train_df.columns if train_df[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 5 numerical features : ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Listening_Time_minutes']

We have 6 categorical features : ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']


In [11]:
import pandas as pd

def remove_outliers_iqr(df, columns):
    """
    Removes outliers from specified columns using the IQR method.
    Returns a new DataFrame with outliers removed.
    """
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean
train_df_clean = remove_outliers_iqr(train_df, numeric_features)

In [12]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# def plot_boxplots(df, columns):
#     """
#     Plots boxplots for the specified columns in the DataFrame.
#     """
#     plt.figure(figsize=(15, 5 * len(columns)))
#     for i, col in enumerate(columns):
#         plt.subplot(len(columns), 1, i + 1)
#         sns.boxplot(x=df[col])
#         plt.title(f'Boxplot for {col}')
#     plt.tight_layout()
#     plt.show()


In [13]:
# plot_boxplots(train_df_clean, numeric_features)

In [14]:
from sklearn.preprocessing import LabelEncoder
import pickle

# Fit encoders on both train and test combined (to avoid unseen values)
podcast_le = LabelEncoder()
title_le = LabelEncoder()

podcast_le.fit(train_df_clean['Podcast_Name'])
title_le.fit(train_df_clean['Episode_Title'])

train_df_clean['Podcast_ID'] = podcast_le.transform(train_df_clean['Podcast_Name'])
train_df_clean['Title_ID'] = title_le.transform(train_df_clean['Episode_Title'])

with open("podcast_id_le.pkl", "wb") as f:
    pickle.dump(podcast_le, f)

with open("episode_title_le.pkl", "wb") as f:
    pickle.dump(title_le, f)



In [15]:
train_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749990 entries, 0 to 749999
Data columns (total 13 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 749990 non-null  object 
 1   Episode_Title                749990 non-null  object 
 2   Episode_Length_minutes       749990 non-null  float64
 3   Genre                        749990 non-null  object 
 4   Host_Popularity_percentage   749990 non-null  float64
 5   Publication_Day              749990 non-null  object 
 6   Publication_Time             749990 non-null  object 
 7   Guest_Popularity_percentage  749990 non-null  float64
 8   Number_of_Ads                749990 non-null  float64
 9   Episode_Sentiment            749990 non-null  object 
 10  Listening_Time_minutes       749990 non-null  float64
 11  Podcast_ID                   749990 non-null  int64  
 12  Title_ID                     749990 non-null  int64  
dtypes: f

In [16]:
train_df_clean.head(10)

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Podcast_ID,Title_ID
0,Mystery Matters,Episode 98,63.84,True Crime,74.81,Thursday,Night,53.58,0.0,Positive,31.41998,34,98
1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241,24,19
2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531,40,8
3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824,10,40
4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031,31,85
5,Fitness First,Episode 19,26.54,Health,48.96,Saturday,Afternoon,53.58,3.0,Positive,22.77047,14,11
6,Criminal Minds,Episode 47,69.83,True Crime,35.82,Sunday,Night,39.02,0.0,Neutral,64.75024,6,42
7,News Roundup,Episode 44,48.52,News,44.99,Thursday,Night,20.12,0.0,Positive,22.37517,35,39
8,Daily Digest,Episode 32,105.87,News,69.81,Monday,Evening,53.58,2.0,Neutral,68.00124,8,26
9,Music Matters,Episode 81,63.84,Music,82.18,Thursday,Night,59.72,3.0,Neutral,45.94761,33,80


In [17]:

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
cat_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column in the list
for col in ['Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']:
    train_df_clean[col] = cat_encoder.fit_transform(train_df_clean[col])


with open("other_cat_vals.pkl", "wb") as f:
    pickle.dump(title_le, f)




In [18]:
train_df_clean.head(10)

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,Podcast_ID,Title_ID
0,Mystery Matters,Episode 98,63.84,9,74.81,4,3,53.58,0.0,2,31.41998,34,98
1,Joke Junction,Episode 26,119.80,1,66.95,2,0,75.95,2.0,0,88.01241,24,19
2,Study Sessions,Episode 16,73.90,2,69.97,5,1,8.97,0.0,0,44.92531,40,8
3,Digital Digest,Episode 45,67.17,8,57.22,1,2,78.70,2.0,2,46.27824,10,40
4,Mind & Body,Episode 86,110.51,3,80.07,1,0,58.68,3.0,1,75.61031,31,85
5,Fitness First,Episode 19,26.54,3,48.96,2,0,53.58,3.0,2,22.77047,14,11
6,Criminal Minds,Episode 47,69.83,9,35.82,3,3,39.02,0.0,1,64.75024,6,42
7,News Roundup,Episode 44,48.52,6,44.99,4,3,20.12,0.0,2,22.37517,35,39
8,Daily Digest,Episode 32,105.87,6,69.81,1,1,53.58,2.0,1,68.00124,8,26
9,Music Matters,Episode 81,63.84,5,82.18,4,3,59.72,3.0,1,45.94761,33,80


In [19]:
train_df_clean.drop(columns=['Podcast_Name', 'Episode_Title'], inplace=True)

In [20]:
train_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 749990 entries, 0 to 749999
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Episode_Length_minutes       749990 non-null  float64
 1   Genre                        749990 non-null  int64  
 2   Host_Popularity_percentage   749990 non-null  float64
 3   Publication_Day              749990 non-null  int64  
 4   Publication_Time             749990 non-null  int64  
 5   Guest_Popularity_percentage  749990 non-null  float64
 6   Number_of_Ads                749990 non-null  float64
 7   Episode_Sentiment            749990 non-null  int64  
 8   Listening_Time_minutes       749990 non-null  float64
 9   Podcast_ID                   749990 non-null  int64  
 10  Title_ID                     749990 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 68.7 MB


In [21]:
train,test=train_test_split(train_df_clean,test_size=0.25,random_state=42)

In [22]:
train_x=train.drop(['Listening_Time_minutes'],axis=1).values
train_y = train[['Listening_Time_minutes']].values.ravel()

##test dataset
test_x=test.drop(['Listening_Time_minutes'],axis=1).values
test_y=test[['Listening_Time_minutes']].values.ravel()

## splitting this train data into train and validation
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)


signature = infer_signature(train_x,train_y)

In [23]:
## ANN MOdel

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture 
    mean= np.mean(train_x,axis=0)
    var= np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Track the ANN model with lr and momentum prams with MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                    epochs=epochs,
                    batch_size=64)
        
        ##Evaluate the model
        eval_result = model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse = eval_result[1]


        #Log the parameters and results

        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ##log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)
        return {"loss":eval_rmse,"status":STATUS_OK,"model":model}




In [24]:
def objective(params):
    #Mlflow will track the parameters and results for each run 
    result = train_model(
        params,
        epochs=10,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [25]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [26]:
mlflow.set_experiment("predict_podcast_time")
with mlflow.start_run():
    #Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    #Fetch the details of the best run 
    best_run = sorted( trials.results, key=lambda x: x["loss"])[0]

    #log the best parameters,loss and model

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)


    #Print out the best parameters and corresponding loss
    print(f"Best Parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/04/25 17:36:18 INFO mlflow.tracking.fluent: Experiment with name 'predict_podcast_time' does not exist. Creating a new experiment.


Epoch 1/10                                           

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

E0000 00:00:1745582778.394325  472562 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1745582778.394559  472562 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


   1/7032 ━━━━━━━━━━━━━━━━━━━━ 23:31 201ms/step - loss: 2838.5554 - root_mean_squared_error: 53.2781
  90/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 562us/step - loss: 1061.6670 - root_mean_squared_error: 31.2524   
 198/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 509us/step - loss: 693.3788 - root_mean_squared_error: 24.9124 
  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

 285/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 531us/step - loss: 574.8584 - root_mean_squared_error: 22.6387
 384/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 525us/step - loss: 497.2563 - root_mean_squared_error: 21.0683
 466/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 541us/step - loss: 454.8906 - root_mean_squared_error: 20.1791
 543/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 557us/step - loss: 425.2126 - root_mean_squared_error: 19.5399
 651/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 542us/step - loss: 394.1721 - root_mean_squared_error: 18.8582
 759/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 531us/step - loss: 370.9600 - root_mean_squared_error: 18.3376
 864/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 524us/step - loss: 353.3805 - root_mean_squared_error: 17.9373
 975/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 516us/step - loss: 338.4104 - root_mean_squared_error: 17.5910
1082/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 511us/step - loss: 326.5703 - root_mean_squared_error: 17.3138
1157/7032 ━━━━━━━━━━━━━━━━━━━━ 3s 521us/step - loss: 319.4352 - root_mean_squared_error: 17.1453
1245/7032 ━━━━━━━━━━━━━━━━━━━━